In [7]:
!pip install torch
!pip install torch_geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 733.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.4 MB/s eta 0:00:0000:0100:01


In [8]:
!pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 975.7 kB/s eta 0:00:000:0100:01m


In [9]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/twibot20-format22/Twibot-20/user_info.pt
/kaggle/input/twibot20-format22/Twibot-20/node.json
/kaggle/input/twibot20-format22/Twibot-20/test.ipynb
/kaggle/input/twibot20-format22/Twibot-20/label.csv
/kaggle/input/twibot20-format22/Twibot-20/edge.csv
/kaggle/input/twibot20-format22/Twibot-20/split.csv
/kaggle/input/twibot20/Twibot-20/LICENSE
/kaggle/input/twibot20/Twibot-20/dev.json
/kaggle/input/twibot20/Twibot-20/README.txt
/kaggle/input/twibot20/Twibot-20/train.json
/kaggle/input/twibot20/Twibot-20/test.json
/kaggle/input/twibot20/Twibot-20/support.json
/kaggle/input/twibot20/Twibot-20/TwiBot-20 Seed Users.txt
/kaggle/input/twibot20-processed/processed_data/edge_type.pt
/kaggle/input/twibot20-processed/processed_data/label.pt
/kaggle/input/twibot20-processed/processed_data/des_tensor.pt
/kaggle/input/twibot20-processed/processed_data/edge_index.pt
/kaggle/input/twibot20-processed/processed_data/cat_properties_tensor.pt
/kaggle/input/twibot20-processed/processed_data/twee

In [10]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

edges = pd.read_csv('/kaggle/input/twibot20-format22/Twibot-20/edge.csv')
edges.dropna(inplace=True)

post_edges = edges[edges['relation'] == 'post']
friend_follow_edges = edges[edges['relation'].isin(['friend', 'follow'])]

post_edges.to_csv('post_relationships.csv', index=False)
friend_follow_edges.to_csv('friend_follow_relationships.csv', index=False)

In [31]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
from IPython.display import IFrame

# Load edges
post_edges = pd.read_csv('post_relationships.csv').head(1000)
friend_follow_edges = pd.read_csv('friend_follow_relationships.csv').head(1000)

# Create a NetworkX graph
G_post = nx.Graph()
for index, row in post_edges.iterrows():
    G_post.add_edge(row['source_id'], row['target_id'])

# Calculate betweenness centrality for post graph
betweenness_post = nx.betweenness_centrality(G_post)

# Normalize the centrality values for visualization
min_size = 10
max_size = 100
sizes_post = [min_size + (max_size - min_size) * betweenness_post[node] for node in G_post.nodes()]

# Create a PyVis network for post relationships
net_post = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')
for node, size in zip(G_post.nodes(), sizes_post):
    net_post.add_node(node, size=size)

for index, row in post_edges.iterrows():
    net_post.add_edge(row['source_id'], row['target_id'], color='blue')

net_post.show('post_relationships.html', notebook=False)
display(IFrame('post_relationships.html', width='100%', height='750px'))

# Repeat the process for friend/follow relationships
G_friend_follow = nx.Graph()
for index, row in friend_follow_edges.iterrows():
    G_friend_follow.add_edge(row['source_id'], row['target_id'])

# Calculate betweenness centrality for friend/follow graph
betweenness_friend_follow = nx.betweenness_centrality(G_friend_follow)

# Normalize the centrality values for visualization
sizes_friend_follow = [min_size + (max_size - min_size) * betweenness_friend_follow[node] for node in G_friend_follow.nodes()]

# Create a PyVis network for friend/follow relationships
net_friend_follow = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')
for node, size in zip(G_friend_follow.nodes(), sizes_friend_follow):
    net_friend_follow.add_node(node, size=size)

for index, row in friend_follow_edges.iterrows():
    if row['relation'] == 'friend':
        net_friend_follow.add_edge(row['source_id'], row['target_id'], color='purple')
        net_friend_follow.add_edge(row['target_id'], row['source_id'], color='purple')
    elif row['relation'] == 'follow':
        net_friend_follow.add_edge(row['source_id'], row['target_id'], color='orange')

net_friend_follow.show('friend_follow_relationships.html', notebook=False)
display(IFrame('friend_follow_relationships.html', width='100%', height='750px'))


post_relationships.html


friend_follow_relationships.html


In [28]:
import os
import torch
import os.path as osp
from torch_geometric.data import Data
import numpy as np

# Path to the processed data
path = '/kaggle/input/twibot20-processed/processed_data'

# Load tensors from files
label_tensor = torch.load(osp.join(path, 'label.pt'))
des_tensor = torch.load(osp.join(path, 'des_tensor.pt'))
tweets_tensor = torch.load(osp.join(path, 'tweets_tensor.pt'))
num_properties_tensor = torch.load(osp.join(path, 'num_properties_tensor.pt'))
cat_properties_tensor = torch.load(osp.join(path, 'cat_properties_tensor.pt'))
edge_index = torch.load(osp.join(path, 'edge_index.pt'))

# Convert tensors to float
des_tensor = des_tensor.float()
tweets_tensor = tweets_tensor.float()
num_properties_tensor = num_properties_tensor.float()
cat_properties_tensor = cat_properties_tensor.float()


num_samples = 11826
random_indices = np.random.choice(label_tensor.shape[0], num_samples, replace=False)

label_tensor = label_tensor[random_indices]
des_tensor = des_tensor[random_indices]
tweets_tensor = tweets_tensor[random_indices]
num_properties_tensor = num_properties_tensor[random_indices]
cat_properties_tensor = cat_properties_tensor[random_indices]

# Concatenate all feature tensors to form node features
node_features = torch.cat((des_tensor, tweets_tensor, num_properties_tensor, cat_properties_tensor), dim=1)

# Check for consistency between label tensor and node features
if label_tensor.shape[0] != node_features.shape[0]:
    raise ValueError("label_tensor and node_features must have the same number of samples")

# Print number of unique classes and number of features
num_classes = label_tensor.unique().numel()
num_features = node_features.shape[1]
print("Number of classes:", num_classes)
print("Number of features:", num_features)

# Map the original indices to new indices
index_map = {old_idx: new_idx for new_idx, old_idx in enumerate(random_indices)}
edge_index_mapped = edge_index.clone()

# Apply the mapping to edge_index
for i in range(edge_index.shape[1]):
    src, dst = edge_index[:, i]
    if src.item() in index_map and dst.item() in index_map:
        edge_index_mapped[0, i] = index_map[src.item()]
        edge_index_mapped[1, i] = index_map[dst.item()]
    else:
        edge_index_mapped[:, i] = -1  # Mark invalid edges

# Filter out invalid edges
valid_mask = (edge_index_mapped[0] >= 0) & (edge_index_mapped[1] >= 0)
edge_index_mapped = edge_index_mapped[:, valid_mask]

# Create Data object
data = Data(x=node_features, edge_index=edge_index_mapped, y=label_tensor)

# Print information about the Data object
print(data)
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average number of edges per node: {data.num_edges / data.num_nodes:.2f}')
print(f'Are there any isolated nodes in the graph: {data.has_isolated_nodes()}')
print(f'Are there any self-loops in the graph: {data.has_self_loops()}')
print(f'Is the graph undirected: {data.is_undirected()}')


Number of classes: 2
Number of features: 1544
Data(x=[11826, 1544], edge_index=[2, 16908], y=[11826])
Number of nodes: 11826
Number of edges: 16908
Average number of edges per node: 1.43
Are there any isolated nodes in the graph: True
Are there any self-loops in the graph: False
Is the graph undirected: False


In [37]:
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader

train_size = 5826
val_size = 1000
test_size = data.num_nodes - train_size - val_size

data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

data.train_mask[:train_size] = True
data.val_mask[train_size:train_size + val_size] = True
data.test_mask[train_size + val_size:] = True

In [20]:
import torch
import torch.nn as nn
import os.path as osp
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

class BotGCN(nn.Module):
    def __init__(self, hidden_dim, des_size=768, tweet_size=768, num_prop_size=5, cat_prop_size=3, dropout=0.3):
        super(BotGCN, self).__init__()
        self.linear_relu_des = nn.Sequential(
            nn.Linear(des_size, hidden_dim // 4),
            nn.LeakyReLU()
        )
        self.linear_relu_tweet = nn.Sequential(
            nn.Linear(tweet_size, hidden_dim // 4),
            nn.LeakyReLU()
        )
        self.linear_relu_num_prop = nn.Sequential(
            nn.Linear(num_prop_size, hidden_dim // 4),
            nn.LeakyReLU()
        )
        self.linear_relu_cat_prop = nn.Sequential(
            nn.Linear(cat_prop_size, hidden_dim // 4),
            nn.LeakyReLU()
        )
        self.linear_relu_input = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU()
        )
        self.linear_relu_output1 = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU()
        )
        self.linear_output2 = nn.Linear(hidden_dim, 2)
        self.gcn1 = GCNConv(hidden_dim, hidden_dim)
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, edge_index):
        d = self.linear_relu_des(x[:, :768])
        t = self.linear_relu_tweet(x[:, 768:1536])
        n = self.linear_relu_num_prop(x[:, 1536:1541])
        c = self.linear_relu_cat_prop(x[:, 1541:1544])
        combined = torch.cat((d, t, n, c), dim=1)
        combined = self.dropout(combined)
        combined = self.linear_relu_input(combined)
        combined = self.gcn1(combined, edge_index)
        combined = self.dropout(combined)
        combined = self.gcn2(combined, edge_index)
        combined = self.linear_relu_output1(combined)
        output = self.linear_output2(combined)
        return output

In [21]:
from tqdm import tqdm

model = BotGCN(512)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

for epoch in tqdm(range(101)):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

  1%|          | 1/101 [00:01<02:19,  1.39s/it]

Epoch: 000, Loss: 0.6933


  2%|▏         | 2/101 [00:02<02:05,  1.27s/it]

Epoch: 001, Loss: 10.3921


  3%|▎         | 3/101 [00:03<01:59,  1.22s/it]

...

Epoch: 098, Loss: 0.5376


 99%|█████████▉| 100/101 [01:53<00:01,  1.12s/it]

Epoch: 099, Loss: 0.5256


100%|██████████| 101/101 [01:54<00:00,  1.13s/it]

Epoch: 100, Loss: 0.5357


In [22]:

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def evaluate(mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        y_true = data.y[mask]
        y_pred = pred[mask]

        accuracy = accuracy_score(y_true.cpu(), y_pred.cpu())
        f1 = f1_score(y_true.cpu(), y_pred.cpu(), average='weighted')
        precision = precision_score(y_true.cpu(), y_pred.cpu(), average='weighted')
        recall = recall_score(y_true.cpu(), y_pred.cpu(), average='weighted')

    return accuracy, f1, precision, recall


In [23]:

    print("Test Metrics:")
    test_acc, test_f1, test_precision, test_recall = evaluate(data.test_mask)
    print(f'Test Accuracy: {test_acc:.4f}')
    print(f'Test F1 Score: {test_f1:.4f}')
    print(f'Test Precision: {test_precision:.4f}')
    print(f'Test Recall: {test_recall:.4f}')

Test Metrics:
Test Accuracy: 0.7154
Test F1 Score: 0.7086
Test Precision: 0.7181
Test Recall: 0.7154


In [24]:
import torch
import torch.nn as nn
import os.path as osp
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from tqdm import tqdm

class BotGAT(nn.Module):
    def __init__(self, hidden_dim, des_size=768, tweet_size=768, num_prop_size=5, cat_prop_size=3, dropout=0.3):
        super(BotGAT, self).__init__()
        self.linear_relu_des = nn.Sequential(
            nn.Linear(des_size, hidden_dim // 4),
            nn.LeakyReLU()
        )
        self.linear_relu_tweet = nn.Sequential(
            nn.Linear(tweet_size, hidden_dim // 4),
            nn.LeakyReLU()
        )
        self.linear_relu_num_prop = nn.Sequential(
            nn.Linear(num_prop_size, hidden_dim // 4),
            nn.LeakyReLU()
        )
        self.linear_relu_cat_prop = nn.Sequential(
            nn.Linear(cat_prop_size, hidden_dim // 4),
            nn.LeakyReLU()
        )

        self.linear_relu_input = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU()
        )
        self.linear_relu_output1 = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU()
        )
        self.linear_output2 = nn.Linear(hidden_dim, 2)

        self.gat1 = GATConv(hidden_dim, hidden_dim // 4, heads=4)
        self.gat2 = GATConv(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, des, tweet, num_prop, cat_prop, edge_index, edge_type=None):
        d = self.linear_relu_des(des)
        t = self.linear_relu_tweet(tweet)
        n = self.linear_relu_num_prop(num_prop)
        c = self.linear_relu_cat_prop(cat_prop)
        x = torch.cat((d, t, n, c), dim=1)
        x = self.dropout(x)
        x = self.linear_relu_input(x)
        x = self.gat1(x, edge_index)
        x = self.dropout(x)
        x = self.gat2(x, edge_index)
        x = self.linear_relu_output1(x)
        x = self.linear_output2(x)
        return x

In [25]:
des = data.x[:, :768]
tweet = data.x[:, 768:1536]
num_prop = data.x[:, 1536:1541]
cat_prop = data.x[:, 1541:]


model = BotGAT(512)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(des, tweet, num_prop, cat_prop, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(mask):
    model.eval()
    with torch.no_grad():
        out = model(des, tweet, num_prop, cat_prop, data.edge_index)
        pred = out.argmax(dim=1)
        y_true = data.y[mask]
        y_pred = pred[mask]

        accuracy = accuracy_score(y_true.cpu(), y_pred.cpu())
        f1 = f1_score(y_true.cpu(), y_pred.cpu(), average='weighted')
        precision = precision_score(y_true.cpu(), y_pred.cpu(), average='weighted')
        recall = recall_score(y_true.cpu(), y_pred.cpu(), average='weighted')

    return accuracy, f1, precision, recall


In [26]:
for epoch in tqdm(range(101)):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')


  1%|          | 1/101 [00:01<02:25,  1.46s/it]

Epoch: 000, Loss: 0.7011


  2%|▏         | 2/101 [00:02<02:13,  1.35s/it]

Epoch: 001, Loss: 3.6094


  3%|▎         | 3/101 [00:04<02:10,  1.33s/it]

...

Epoch: 098, Loss: 0.5285


 99%|█████████▉| 100/101 [02:08<00:01,  1.36s/it]

Epoch: 099, Loss: 0.5260


100%|██████████| 101/101 [02:09<00:00,  1.28s/it]

Epoch: 100, Loss: 0.5306


In [27]:
    print("Validation Metrics:")
    val_acc, val_f1, val_precision, val_recall = evaluate(data.val_mask)
    print(f'Validation Accuracy: {val_acc:.4f}')
    print(f'Validation F1 Score: {val_f1:.4f}')
    print(f'Validation Precision: {val_precision:.4f}')
    print(f'Validation Recall: {val_recall:.4f}')

    print("Test Metrics:")
    test_acc, test_f1, test_precision, test_recall = evaluate(data.test_mask)
    print(f'Test Accuracy: {test_acc:.4f}')
    print(f'Test F1 Score: {test_f1:.4f}')
    print(f'Test Precision: {test_precision:.4f}')
    print(f'Test Recall: {test_recall:.4f}')

Validation Metrics:
Validation Accuracy: 0.7450
Validation F1 Score: 0.7418
Validation Precision: 0.7478
Validation Recall: 0.7450
Test Metrics:
Test Accuracy: 0.7436
Test F1 Score: 0.7410
Test Precision: 0.7433
Test Recall: 0.7436


In [38]:
import torch
import torch.nn as nn
from torch_geometric.nn import SAGEConv

class BotGraphSAGE(nn.Module):
    def __init__(self, hidden_dim, des_size=768, tweet_size=768, num_prop_size=5, cat_prop_size=3, dropout=0.3):
        super(BotGraphSAGE, self).__init__()
        self.linear_relu_des = nn.Sequential(
            nn.Linear(des_size, hidden_dim // 4),
            nn.LeakyReLU()
        )
        self.linear_relu_tweet = nn.Sequential(
            nn.Linear(tweet_size, hidden_dim // 4),
            nn.LeakyReLU()
        )
        self.linear_relu_num_prop = nn.Sequential(
            nn.Linear(num_prop_size, hidden_dim // 4),
            nn.LeakyReLU()
        )
        self.linear_relu_cat_prop = nn.Sequential(
            nn.Linear(cat_prop_size, hidden_dim // 4),
            nn.LeakyReLU()
        )
        self.linear_relu_input = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU()
        )
        self.linear_relu_output1 = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU()
        )
        self.linear_output2 = nn.Linear(hidden_dim, 2)
        self.sage1 = SAGEConv(hidden_dim, hidden_dim)
        self.sage2 = SAGEConv(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, edge_index):
        d = self.linear_relu_des(x[:, :768])
        t = self.linear_relu_tweet(x[:, 768:1536])
        n = self.linear_relu_num_prop(x[:, 1536:1541])
        c = self.linear_relu_cat_prop(x[:, 1541:1544])
        combined = torch.cat((d, t, n, c), dim=1)
        combined = self.dropout(combined)
        combined = self.linear_relu_input(combined)
        combined = self.sage1(combined, edge_index)
        combined = self.dropout(combined)
        combined = self.sage2(combined, edge_index)
        combined = self.linear_relu_output1(combined)
        output = self.linear_output2(combined)
        return output

In [39]:
from tqdm import tqdm

# Initialize the model, optimizer, and loss function
model = BotGraphSAGE(512)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# Train the model
for epoch in tqdm(range(101)):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

  1%|          | 1/101 [00:01<02:47,  1.68s/it]

Epoch: 000, Loss: 0.6916


  2%|▏         | 2/101 [00:03<02:48,  1.70s/it]

Epoch: 001, Loss: 4.5031


  3%|▎         | 3/101 [00:06<03:46,  2.31s/it]

...

Epoch: 098, Loss: 0.5902


 99%|█████████▉| 100/101 [03:08<00:01,  1.23s/it]

Epoch: 099, Loss: 0.5895


100%|██████████| 101/101 [03:09<00:00,  1.88s/it]

Epoch: 100, Loss: 0.5781


In [42]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def evaluate(mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        y_true = data.y[mask]
        y_pred = pred[mask]

        accuracy = accuracy_score(y_true.cpu(), y_pred.cpu())
        f1 = f1_score(y_true.cpu(), y_pred.cpu(), average='weighted')
        precision = precision_score(y_true.cpu(), y_pred.cpu(), average='weighted')
        recall = recall_score(y_true.cpu(), y_pred.cpu(), average='weighted')

    return accuracy, f1, precision, recall

# Example of evaluating on a validation or test mask
test_mask = data.test_mask
accuracy, f1, precision, recall = evaluate(test_mask)
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test F1 Score: {f1:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test Recall: {recall:.4f}')

Test Accuracy: 0.7292
Test F1 Score: 0.7240
Test Precision: 0.7322
Test Recall: 0.7292
